In [ ]:
import numpy as np
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import urllib.request
import shutil
import os
import random
from copy import deepcopy
import pprint as pp

from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader
import nltk.data
from nltk import sent_tokenize, pos_tag, wordpunct_tokenize
import en_core_web_lg
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
# https://github.com/buriy/python-readability
from readability.readability import Unparseable
from readability.readability import Document as Paper

# https://docs.python.org/3/library/time.html
import time

# https://beautiful-soup-4.readthedocs.io/en/latest/
import bs4

# https://docs.python.org/3/library/codecs.html
import codecs

# https://docs.python.org/3/library/json.html
import json

import os

from time import time

import re

import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from time import time  # To time our operations

from tqdm import tqdm

Load the 1990-1994 vs 1995-1999

In [409]:
#%%
# we create a list to contain the json files that are to be processed
# DOC_ID = ['mmsu_art_en_html.json']
#year = 2020
#DOC_ID=list()
#for x in os.listdir(r"C:/_harvester/data/html-by-year/20s"):
#    if x.endswith(".json") and x[:4] in ['2020']:
#        DOC_ID.append(x)
#print(DOC_ID)
DOC_ID =r'.*([\d]+_html\.json)'


# we create a list of categories/keywords/tags to
#cat_pattern = r'(.*)[\d]_html\.json'
#cat_pattern = r'(.*?)(\d{,4}?_html\.json'
#cat_pattern = r'(.*?)\d{,4}\.[\w]+'
CAT_PATTERN = r'^(.*?)[\W]*?\d{,4}?_html\.json'
#CAT_PATTERN = None


# we mark the HTML tags to be used for 
# extacting the desired article, etc. text
# don't include 'li' tag e.g. <li>The Centers for Disease Control and Prevention (CDC) cannot attest to the accuracy of a non-federal website.</li>
TAGS = ['p']

# stop words
stop_words = stopwords.words('english')
# observed findings 
stop_words.extend(['icon', 'website', 'mmwr', 'citation', 'author', 'report', 'formatting', "format",'regarding',
                   'system', 'datum', 'link', 'linking', 'federal', 'data', 'tract', 'census', 'study',"question",
                   'conduct', 'report', 'including', 'top', 'summary', 'however', 'name', 'known',
                   'page', 'view', 'affiliation', 'pdf', 'law', 'version', 'list', 'endorsement', "review",
                   'article', 'download', 'reference', 'publication', 'discussion', 'table', 'vol', "message",
                   'information', 'web', 'notification', 'policy', 'policie', #spaCy lemmatization can make errors with pluralization(e.g. rabie for rabies)
                   'acknowledgment', 'altmetric',
                   'abbreviation', 'figure', "service","imply","current","source",
                   "trade","address", "addresses","program","organization" ,"provided", "copyrighted", "copyright",
                   "already", "topic", "art"])

# pretrained model for POS tagging/filtering
nlp = en_core_web_lg.load( disable=['parser','ner'])

# set encoding for CorpusReader class
ENCODING = 'utf8'

# set the minimum number of topics to find
MIN_TOPICS = 25

# set the maximum number of topics to find
MAX_TOPICS = 65

# set the step by value
STEP_BY = 5

# set value to determine if lemmatization will be performed
LEMMATIZATION = True

In [395]:

"""
Applied Text Analysis with Python by Benjamin Bengfort, Rebecca Bilbro, 
and Tony Ojeda(O'Reilly). 978-1-491-96304-3
"""

class JOURNALCorpusReader(CategorizedCorpusReader, CorpusReader):
    """ a corpus reader for CDC Journal articles """
    # class nltk.corpus.reader.api.CorpusReader
    # we explicitly specify the encoding as utf8 even though
    # the default is utf8
    def __init__(self, root, tags=TAGS, fileids=DOC_ID, encoding=ENCODING, **kwargs):
            
        # we use this check to see if the user specified any
        # values in the CAT_PATTERN list
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN
            #pp.pprint(kwargs)

        # initialize the NLTK  reader objects
        # review https://www.nltk.org/api/nltk.corpus.reader.api.html#nltk.corpus.reader.api.CategorizedCorpusReader to see
        # how __init__ is defined for each module; for the categorized
        # corpus reader, we use it to create categories if none are specified.
        CategorizedCorpusReader.__init__(self, kwargs)

        # https://www.nltk.org/api/nltk.corpus.reader.api.html#nltk.corpus.reader.api.CorpusReader
        # encoding –
        # The default unicode encoding for the files that make up the corpus. The value of encoding can 
        # be any of the following:
        #   A string: encoding is the encoding name for all files.
        #   A dictionary: encoding[file_id] is the encoding name for the file whose identifier is file_id. If file_id is not in encoding, 
        #       then the file contents will be processed using non-unicode byte strings.
        #   A list: encoding should be a list of (regexp, encoding) tuples. The encoding for a file whose 
        #       identifier is file_id will be the encoding value for the first tuple whose regexp matches
        #        the file_id. If no tuple’s regexp matches the file_id, the file contents will be processed using non-unicode byte strings.
        #   None: the file contents of all files will be processed using non-unicode byte strings.
        CorpusReader.__init__(self, root, fileids, encoding)
        
        #self.fileids = fileids
        #self.categories = self.categories()
        self.tags = tags

        #print("From the constructor these are the fileids", fileids)
        #print("from the constructor these are the categories", self.categories)

    # we create a method that will allow us to filter how we
    # read the data from disk, either by specifying a list of categories
    # or a list of filenames
    def resolve(self, fileids=None, categories=None):
        #if fileids is not None and categories is not None:
        #   raise ValueError("Specify fileids or categories, not both")
            
        if categories is not None:
            #pp.pprint("This is a test of the resolve() method where categories is not None:")
            #pp.pprint(categories)
            #print("\n")

            return self.fileids(categories)
        
        #pp.pprint("This is a test of the resolve() method where fileids is not None:")
        #pp.pprint(fileids)
        #print("\n")

        return fileids

    # we use this method to read all values from the key-value objects,
    # concatenating them into a list object which is returned.
    def docs(self,fileids=None, categories=None):

        fileids = self.resolve(self.fileids(), categories)
        #for f in fileids:
        #    pp.pprint( f)
        
        # https://docs.python.org/3/library/codecs.html
        # This module defines base classes for standard Python codecs 
        # (encoders and decoders) and provides access to the internal Python 
        # codec registry, which manages the codec and error handling 
        # lookup process. Most standard codecs are text encodings, which encode 
        # text to bytes (and decode bytes to text), but there are also codecs 
        # provided that encode text to text, and bytes to bytes. Custom codecs 
        # may encode and decode between arbitrary types, but some module features 
        # are restricted to be used specifically with text encodings or with codecs 
        # that encode to bytes.

        # A string in Python is a sequence of Unicode code points (in range U+0000–U+10FFFF). To store or 
        # transfer a string, it needs to be serialized as a sequence of bytes.
        # Serializing a string into a sequence of bytes is known as “encoding”, 
        # and recreating the string from the sequence of bytes is known as “decoding”.
        # There are a variety of different text serialization codecs, which are collectively 
        # referred to as “text encodings”.

        # codecs.open(filename, mode='r', encoding=None, errors='strict', buffering=-1)
        # Open an encoded file using the given mode and return an instance of StreamReaderWriter, 
        # providing transparent encoding/decoding. The default file mode is 'r', meaning to open 
        # the file in read mode.
        # Note If encoding is not None, then the underlying encoded files are always opened in binary 
        # mode. No automatic conversion of '\n' is done on reading and writing. The mode argument
        # may be any binary mode acceptable to the built-in open() function; 
        # the 'b' is automatically added.
        
        # abspaths() Return a list of the absolute paths for all fileids in this corpus; 
        #       or for the given list of fileids, if specified.
        docbag = list()
        for fileid in fileids:
            for path, encoding in self.abspaths(fileid, include_encoding=True):
                with codecs.open(path, 'r', encoding=encoding) as f:
                    #pp.pprint(path)
                    data = json.load(f)
                    docbag.append(str(data))

        #pp.pprint(docbag)
        return docbag


    # we use this method to to parse the HTML
    def html(self, fileids=None, categories=None):
        for doc in self.docs(fileids, categories):
            try:
                yield Paper(doc).summary() # summer() Given an HTML file, extracts the text of the article
            except Unparseable as e:
                print("Could not parse HTML: {}".format(e))
                continue
            
    def paras(self, fileids=None, categories=None):
        for html in self.html(fileids, categories):
            soup=bs4.BeautifulSoup(html,'html.parser')
            for element in soup.find_all(TAGS):
                yield element.text
            soup.decompose()
                
    def sents(self, fileids=None, categories=None):
        for paragraph in self.paras(fileids, categories):
            for sentence in sent_tokenize(paragraph):
                yield sentence

    def words(self, fileids=None, categories=None):
        for sentence in self.sents(fileids, categories):
            for token in wordpunct_tokenize(sentence):
                yield token
    
    """
    def tokenize(self, fileids=None, categories=None):
        for paragraph in self.paras(fileids, categories):
            yield[
                    pos_tag(wordpunct_tokenize(sent))
                    for sent in sent_tokenize(paragraph)
                    ]    
    """

    def describe(self, fileids=None, categories=None):
        """
        Performs a single pass of the corpus and
        returns a dictionary with a variety of metrics
        concerning the state of the corpus.
        """
        started = time()
        
        #structures to perform counting
        counts = nltk.FreqDist()
        tokens = nltk.FreqDist()
        

        for doc in self.docs():
            counts['docs'] += 1

        for para in tqdm(enumerate(self.paras()), desc='loading paras()', total=len(list(enumerate(self.paras())))):
            counts['paras'] += 1
                
        for sent in self.sents():
            counts['sents'] += 1
                    
        for word in self.words():
            counts['words'] += 1
            tokens[word] += 1
                    



        #compute  the number of files and categories in the corpus
        n_fileids = len(self.resolve(self.fileids(), categories) or self.fileids())
        #n_fileids = len(fileids)
        #n_topics = len(self.categories(self.resolve(fileids, categories)) or self.categories)
        #n_topics = len(categories)
        
        #return data structure with information
        return{
                'files': self.fileids(),
                # number of files
                'nfiles': n_fileids,
                # number of topics
                #'topics': n_topics,
                # number of docs
                'docs': counts['docs'], 
                # number of paragraphs
                'paras': counts['paras'],
                # number of sentences
                'sents': counts['sents'],
                # number of words
                'words': counts['words'],
                # size of vocabulary ie number of unique terms
                'vocab': len(tokens),
                # lexical diversity, the ratio of unique terms to total words
                'lexdiv': float(counts['words']) / float(len(tokens)),
                # average number of words per sentence
                'wdps': float(counts['words']) / float(counts['sents']),
                # average number of paragraphs per document
                'ppdoc': float(counts['paras']) / float(n_fileids),
                # average number of sentences per paragraph
                'sppar': float(counts['sents']) / float(counts['paras']),
                # total processing time
                'mins': round((time() - started) / 60, 2)
                }


In [396]:
def w2v_to_numpy (model):
  """ Convert the word2vec model (the embeddings) into numpy arrays.
  Also create and return the mapping of words to the row numbers.

  Parameters:
  ===========
  model (gensim.Word2Vec): a trained gensim model

  Returns:
  ========
  embeddings (numpy.ndarray): Embeddings of each word
  idx, iidx (tuple): idx is a dictionary mapping word to row number
                     iidx is a dictionary mapping row number to word
  """
  embeddings = deepcopy (model.wv.get_normed_vectors())
  idx = {w:i for i, w in enumerate (model.wv.index_to_key )}
  iidx = {i:w for i, w in enumerate (model.wv.index_to_key )}
  return embeddings, (idx, iidx)

In [397]:
def near_neighbors (embs, query, word2rownum, rownum2word, k=5):
  """ Get the `k` nearest neighbors for a `query`

  Parameters:
  ===========
  embs (numpy.ndarray): The embeddings.
  query (str): Word whose nearest neighbors are being found
  word2rownum (dict): Map word to row number in the embeddings array
  rownum2word (dict): Map rownum from embeddings array to word
  k (int, default=5): The number of nearest neighbors

  Returns:
  ========
  neighbors (list): list of near neighbors;
                    size of the list is k and each item is in the form
                    of word and similarity.
  """

  sims = np.dot (embs, embs[word2rownum[query]])
  indices = np.argsort (-sims)
  return [(rownum2word[index], sims[index]) for index in indices[1:k+1]]

In [398]:
def procrustes(A, B):
    """
    Learn the best rotation matrix to align matrix B to A
    https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    """
    # U, _, Vt = np.linalg.svd(B.dot(A.T))
    U, _, Vt = np.linalg.svd(B.T.dot(A))
    return U.dot(Vt)

def intersect_vocab (idx1, idx2):
  """ Intersect the two vocabularies

  Parameters:
  ===========
  idx1 (dict): the mapping for vocabulary in the first group
  idx2 (dict): the mapping for vocabulary in the second group

  Returns:
  ========
  common_idx, common_iidx (tuple): the common mapping for vocabulary in both groups
  """
  common = idx1.keys() & idx2.keys()
  common_vocab = [v for v in common]

  common_idx, common_iidx = {v:i for i,v in enumerate (common_vocab)}, {i:v for i,v in enumerate (common_vocab)}
  return common_vocab, (common_idx, common_iidx)

def align_matrices (mat1, mat2, idx1, idx2):
  """ Align the embedding matrices and their vocabularies.

  Parameters:
  ===========
  mat1 (numpy.ndarray): embedding matrix for first group
  mat2 (numpy.ndarray): embedding matrix for second group

  index1 (dict): the mapping dictionary for first group
  index2 (dict): the mapping dictionary for the second group

  Returns:
  ========
  remapped_mat1 (numpy.ndarray): the aligned matrix for first group
  remapped_mat2 (numpy.ndarray): the aligned matrix for second group
  common_vocab (tuple): the mapping dictionaries for both the matrices
  """
  common_vocab, (common_idx, common_iidx) = intersect_vocab (idx1, idx2)
  row_nums1 = [idx1[v] for v in common_vocab]
  row_nums2 = [idx2[v] for v in common_vocab]

  #print (len(common_vocab), len (common_idx), len (common_iidx))
  remapped_mat1 = mat1[row_nums1, :]
  remapped_mat2 = mat2[row_nums2, :]
  #print (mat1.shape, mat2.shape, remapped_mat1.shape, remapped_mat2.shape)
  omega = procrustes (remapped_mat1, remapped_mat2)
  #print (omega.shape)
  # rotated_mat2 = np.dot (omega, remapped_mat2)
  rotated_mat2 = np.dot (remapped_mat2, omega)

  return remapped_mat1, rotated_mat2, (common_idx, common_iidx)

In [407]:
corpus90s = JOURNALCorpusReader("C:/_harvester/data/html-by-year/90s")
corpus00s = JOURNALCorpusReader("C:/_harvester/data/html-by-year/00s")



In [408]:
print(corpus90s.fileids())
print(corpus00s.fileids())
print("\n")

['1990_html.json', '1991_html.json', '1992_html.json', '1993_html.json', '1994_html.json', '1995_html.json', '1996_html.json', '1997_html.json', '1998_html.json', '1999_html.json']
['2000_html.json', '2001_html.json', '2002_html.json', '2003_html.json', '2004_html.json', '2005_html.json', '2006_html.json', '2007_html.json', '2008_html.json', '2009_html.json']




In [406]:
corpus00s.describe()

{'files': ['2000_html.json',
  '2001_html.json',
  '2002_html.json',
  '2003_html.json',
  '2004_html.json',
  '2005_html.json',
  '2006_html.json',
  '2007_html.json',
  '2008_html.json',
  '2009_html.json'],
 'nfiles': 10,
 'docs': 10,
 'paras': 8481,
 'sents': 24417,
 'words': 594974,
 'vocab': 24612,
 'lexdiv': 24.17414269462051,
 'wdps': 24.36720317811361,
 'ppdoc': 848.1,
 'sppar': 2.8790237000353733,
 'mins': 5.94}

### 1990s
{'files': ['1990_html.json', 
  '1991_html.json',
  '1992_html.json',
  '1993_html.json',
  '1994_html.json',
  '1995_html.json',
  '1996_html.json',
  '1997_html.json',
  '1998_html.json',
  '1999_html.json'] \
 'nfiles': 10,\
 'docs': 10,\
 'paras': 15034,\
 'sents': 21145,\
 'words': 276431,\
 'vocab': 15607,\
 'lexdiv': 17.7119882104184,\
 'wdps': 13.073114211397494,\
 'ppdoc': 1503.4,\
 'sppar': 1.406478648396967,\
 'mins': 2.41}

 ### 2000s
 {'files': ['2000_html.json',
  '2001_html.json',
  '2002_html.json',
  '2003_html.json',
  '2004_html.json',
  '2005_html.json',
  '2006_html.json',
  '2007_html.json',
  '2008_html.json',
  '2009_html.json']\
 'nfiles': 10,\
 'docs': 10,\
 'paras': 8481,\
 'sents': 24417,\
 'words': 594974,\
 'vocab': 24612,\
 'lexdiv': 24.17414269462051,\
 'wdps': 24.36720317811361,\
 'ppdoc': 848.1,\
 'sppar': 2.8790237000353733,\
 'mins': 5.94}

In [414]:
from tqdm import tqdm
t = time()

texts_out= list()
inner_text = list()

# number of stopwords found
stopword_count = nltk.FreqDist()

for idx, paras in tqdm(enumerate(corpus00s.paras()), desc='tokenizing sentence content', total=len(list(enumerate(corpus00s.paras())))):
    doc = nlp(paras)
    for token in doc:
        if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']:
            if len(token.text) > 1:
                #pp.pprint(token.text)
                #spaCy can make errors with lemmatization so we check using lemma_ and not using lemma_
                if token.text.lower() not in stop_words and token.lemma_.lower() not in stop_words: 
                    if LEMMATIZATION == False:
                        # don't use lemma_ so as to allow robust contextualization(e.g. Black race versus black)
                        inner_text.append(token.text) 
                    else:
                        inner_text.append(token.lemma_) 
                else:
                    if LEMMATIZATION == False:
                        stopword_count[token.text]+=1
                    else:
                        stopword_count[token.lemma_]+=1

            #pp.pprint(texts_out) 
    if len(inner_text) > 0:
        texts_out.append(inner_text)
    inner_text = []

# return table of FreqDist object
stopword_count.tabulate()
print("\n")

pp.pprint('Time to finish spaCy filter: {} mins'.format(round((time() - t) / 60, 2)))

#pp.pprint(texts_out[0:25])

# tokenizing sentence content: 100%|██████████| 23883/23883 [02:35<00:00, 153.49it/s]

tokenizing sentence content: 100%|██████████| 8481/8481 [02:28<00:00, 57.17it/s] 

       other        study       report      however         more      service        datum      program      include         such         most         only  information       system       regard           as         same          art      conduct           up notification      provide         know       source       review      current       policy           at        tract      address         have         once          few         then organization          law           re      further         name         link        table     question         Most           no       figure         list      federal      version           do   discussion          own          far      already    reference           so      message         very  publication        topic          too      website        above         just          now         page         More        known      article       before         mmwr        again          don           be          top       format          pdf        imply

In [415]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(texts_out, min_count=20)

# freqDist object for bigrams
bigram_freq = nltk.FreqDist()

# print bigrams
for ngrams, _ in bigram.vocab.items():
    #unicode_ngrams = ngrams.decode('utf-8')
    if '_' in ngrams:
        bigram_freq[ngrams]+=1
        print(ngrams)

# add bigrams to texts_out to be included in corpus
for idx in range(len(texts_out)):
    for token in bigram[texts_out[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            texts_out[idx].append(token)

person_use
use_assistive
assistive_technology
technology_able
able_fully
fully_access
access_file
file_assistance
assistance_send
send_mail
mail_type
type_title
title_subject
subject_line
line_mail
note_publish
publish_erratum
a._director\ndivision
director\ndivision_infectious
m.d.\ndivision_infectious
infectious_diseases\ndivision
diseases\ndivision_surveillance
surveillance_tb
a._m.d.\ndivision
surveillance_cdc\n
m.d.\ndivision_surveillance
m.d.\nmemorial_york\n
new_hampshire\n
cosponsor_prevent
prevent_opportunistic
opportunistic_infection
infection_oi
oi_hematopoietic
hematopoietic_transplant
transplant_recipient
recipient_guideline
guideline_draft
draft_assistance
assistance_work
work_group
group_expert
expert_infectious
infectious_disease
disease_transplantation
transplantation_purpose
purpose_define
define_transplantation
transplantation_marrow
marrow_derive
derive_hematopoietic
hematopoietic_stem
stem_cell
cell_regardless
regardless_transplant
transplant_allogeneic
allogeneic_

In [412]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
models = list ()
sliding_window = [5, 10, 15, 20, 25]
for w in sliding_window:
    w2v_90s = Word2Vec( window=w, # Maximum distance between the current and predicted word within a sentence.
                        sample=6e-5, 
                        alpha=0.03, 
                        min_alpha=0.0007, 
                        negative=15,
                        workers=cores-1)
    
    # Create Dictionary
    id2word90s = Dictionary(texts_out)

    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    id2word90s.filter_extremes(no_below=50, no_above=0.5)

    # Bag-of-words representation of the documents.
    corpus90s = [id2word90s.doc2bow(text) for text in texts_out]

    w2v_90s.build_vocab(texts_out, progress_per=100)

    w2v_90s.train(texts_out, total_examples=w2v_90s.corpus_count, epochs=30, report_delay=1)
    models.append(w2v_90s)

In [413]:
query = 'virus'
for i in range (len (sliding_window)):
  window_size = sliding_window[i]
  model = models[i]
  print (f'Model context window size={window_size}')
  for item in model.wv.most_similar(query, topn=5):
    print (item)
  print ()

Model context window size=5
('rodent', 0.9812109470367432)
('import', 0.9783017635345459)
('dengue', 0.9776748418807983)
('indigenous', 0.9775320291519165)
('due', 0.9758689999580383)

Model context window size=10
('human', 0.9557136297225952)
('dengue', 0.9490786790847778)
('united\\nstate', 0.9472452402114868)
('rodent', 0.9442910552024841)
('import', 0.9413126111030579)

Model context window size=15
('rodent', 0.9436047673225403)
('united\\nstate', 0.9297291040420532)
('import', 0.9214004874229431)
('indigenous', 0.9205991625785828)
('influenzae', 0.9035131335258484)

Model context window size=20
('import', 0.9378324747085571)
('rodent', 0.9299533367156982)
('indigenous', 0.9295280575752258)
('united\\nstate', 0.9235057830810547)
('pulmonary', 0.9204021692276001)

Model context window size=25
('united\\nstate', 0.9309801459312439)
('import', 0.9204389452934265)
('rodent', 0.919052243232727)
('indigenous', 0.9163351655006409)
('pulmonary', 0.9113966226577759)



### 1990s
Model context window size=5\
('rodent', 0.9812109470367432)\
('import', 0.9783017635345459)\
('dengue', 0.9776748418807983)\
('indigenous', 0.9775320291519165)\
('due', 0.9758689999580383)\

Model context window size=10\
('human', 0.9557136297225952)\
('dengue', 0.9490786790847778)\
('united\\nstate', 0.9472452402114868)\
('rodent', 0.9442910552024841)\
('import', 0.9413126111030579)\

Model context window size=15\
('rodent', 0.9436047673225403)\
('united\\nstate', 0.9297291040420532)\
('import', 0.9214004874229431)\
('indigenous', 0.9205991625785828)\
('influenzae', 0.9035131335258484)\

Model context window size=20\
('import', 0.9378324747085571)\
('rodent', 0.9299533367156982)\
('indigenous', 0.9295280575752258)\
('united\\nstate', 0.9235057830810547)\
('pulmonary', 0.9204021692276001)\

Model context window size=25\
('united\\nstate', 0.9309801459312439)\
('import', 0.9204389452934265)\
('rodent', 0.919052243232727)\
('indigenous', 0.9163351655006409)\
('pulmonary', 0.9113966226577759)\


In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_2021 = Word2Vec(min_count=20, # (int, optional) – Ignores all words with total frequency lower than this.
                     window=20, # Maximum distance between the current and predicted word within a sentence.
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=15,
                     workers=cores-1)

In [ ]:
# Create Dictionary
id2word2021 = Dictionary(texts2021)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
id2word2021.filter_extremes(no_below=20, no_above=0.5)

# Bag-of-words representation of the documents.
corpus2021 = [id2word2021.doc2bow(text) for text in texts2021]

w2v_2021.build_vocab(texts2021, progress_per=100)

w2v_2021.train(texts2021, total_examples=w2v_2021.corpus_count, epochs=30, report_delay=1)


In [ ]:
w2v_2020.save(f"C:/_harvester/data/word2vec-models/word2vec-2020.model")
w2v_2021.save(f"C:/_harvester/data/word2vec-models/word2vec-2021.model")

In [ ]:
word_list = ['person', 'people', 'virus', 'cough', 'respiratory']
for word in word_list:
    similar_words = w2v_2020.wv.most_similar(positive=[word])
    for word, similarity in similar_words:
        pp.pprint(f"{word}: {similarity}")
    print("\n")

In [ ]:
word_list = ['person', 'people', 'virus', 'cough', 'respiratory']
for word in word_list:
    similar_words = w2v_2021.wv.most_similar(positive=[word])
    for word, similarity in similar_words:
        pp.pprint(f"{word}: {similarity}")
    print("\n")

In [ ]:
# convert embedding to numpy array

"""
  idx, iidx (tuple): idx is a dictionary mapping word to row number
                     iidx is a dictionary mapping row number to word
"""
embs2020, (idx2020, iidx2020) = w2v_to_numpy(w2v_2020)
embs2021, (idx2021, iidx2021) = w2v_to_numpy(w2v_2021)

In [ ]:
query = 'virus'
print (f'Near neighbors for {query} in the 2018 corpus')
for item in near_neighbors (embs2020, query, idx2020, iidx2020, k=10):
  print (item)
print ()
print (f'Near neighbors for {query} in the 2019 corpus')
for item in near_neighbors (embs2021, query, idx2021, iidx2021, k=10):
  print (item)

In [ ]:
_2020_aligned_embs, _2021_aligned_embs, (common_idx, common_iidx) = align_matrices (embs2020, embs2021, idx2020, idx2021)

In [ ]:
near_neighbors(_2020_aligned_embs, 'person', common_idx, common_iidx, k=5)

In [ ]:
near_neighbors(_2021_aligned_embs, 'person', common_idx, common_iidx, k=5)

In [ ]:
journal_words = ['disease','person', 'child', 'adult', 'cough', 'respiratory', 'symptom']
journal_words = [(w, _2020_aligned_embs[common_idx[w]].dot(_2021_aligned_embs[common_idx[w]])) for w in journal_words]
for w,score in sorted (journal_words, key=lambda x:x[1], reverse=True):
  print (w, score)